# propensity score
* 2020/12/20
* in python env, import R dataset doing one hot encoding for race column
* using R kernel in jupyternotebook, example reference from @vlaskinvlad: https://github.com/vlaskinvlad/coursera-causality-crash-course/blob/master/causality_w3.ipynb

In [5]:
library(tableone)
library(Matching)
library(MatchIt)
library(ggplot2)
library(caret)
library(stddiff)
library(smd)

Loading required package: MASS

## 
##  Matching (Version 4.9-7, Build Date: 2020-02-05)
##  See http://sekhon.berkeley.edu/matching for additional documentation.
##  Please cite software as:
##   Jasjeet S. Sekhon. 2011. ``Multivariate and Propensity Score Matching
##   Software with Automated Balance Optimization: The Matching package for R.''
##   Journal of Statistical Software, 42(7): 1-52. 
##


Loading required package: lattice



## Function Pre-define

In [73]:
std_diff <- function(dfr, tcol, gcol='treat'){
    xt <- dfr[dfr[[gcol]]==1,tcol]
    xc <- dfr[dfr[[gcol]]==0,tcol]
    nt = 2; nc = 2;  
    #nt = length(xt);nc = length(xc);
    (mean(xt) - mean(xc))/(sqrt( ((nt-1)*var(xt) + (nc-1)*var(xc))/(nt+nc-2)))
}

## Import data

In [9]:
# Read data
lalonde <- read.csv("lalonde_for_r.csv")

In [11]:
lalonde

treat,age,educ,married,nodegree,re74,re75,re78,black,hispan
<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<int>,<int>
1,37,11,1,1,0,0,9930.0460,1,0
1,22,9,0,1,0,0,3595.8940,0,1
1,30,12,0,0,0,0,24909.4500,1,0
1,27,11,0,1,0,0,7506.1460,1,0
1,33,8,0,1,0,0,289.7899,1,0
1,22,9,0,1,0,0,4056.4940,1,0
1,23,12,0,0,0,0,0.0000,1,0
1,32,11,0,1,0,0,8472.1580,1,0
1,22,16,0,0,0,0,2164.0220,1,0


In [13]:
# Q1 
# standardized differences for all of the confounding variables (pre-matching). What is the standardized difference for married (to nearest hundredth
# smd = (x_1_mean - x_0_mean) / [(S_1^2 + S_0^2)/2]^1/2  -- difference in groups

xt = lalonde[lalonde$treat==1,]$married
xc = lalonde[lalonde$treat==0,]$married
#nt = length(xt);nc = length(xc);
nt = 2; nc = 2;
(mean(xt) - mean(xc))/(sqrt( ((nt-1)*var(xt) + (nc-1)*var(xc))/(nt+nc-2)))

[1] -0.719492

In [14]:
# Q2
# What is the raw (unadjusted) mean of real earnings in 1978 for treated subjects minus the mean of real earnings in 1978 for untreated subjects?
mean(lalonde[lalonde$treat==1,]$re78) - mean(lalonde[lalonde$treat==0,]$re78)

[1] -635.0262

In [15]:
# Q3
# What are the minimum and maximum values of the estimated propensity score?
# Fit a propensity score model. Use a logistic regression model, where the outcome is treatment. 
# Include the 8 confounding variables in the model as predictors, with no interaction terms or non-linear terms (such as squared terms). Obtain the propensity score for each subject.

fit <- glm(treat ~ age + educ + black + hispan + married + nodegree + re74 + re75,
           data=lalonde, family = binomial(link = "logit"))
prop_hat <- predict(fit, newdata = lalonde, type="response")
lalonde$pscore <- prop_hat
min(prop_hat); max(prop_hat)

[1] 0.009080193

[1] 0.8531528

In [76]:
# Q4
# Now carry out propensity score matching using the Match function.
# Match on the propensity score itself, not logit of the propensity score. Obtain the standardized differences for the matched data.
# What is the standardized difference for married? 經過 propensity 匹配之後，有無結婚這個因子與是否接受 treatment 的 stddiff
# 傾向評分匹配: http://www.360doc.com/content/17/0728/02/2856980_674673483.shtml
# 在觀察研究中，由於種種原因，數據偏差（bias）和混雜變量（confounding variable）較多，傾向評分匹配的方法正是為了減少這些偏差和混雜變量的影響，
# 以便對實驗組和對照組進行更合理的比較。

set.seed(931139)
pmatch <- Match(Tr = lalonde$treat, M=1, X=lalonde$pscore, replace=FALSE, caliper=NaN)
# Extract matched data
matched <- lalonde[unlist(pmatch[c('index.treated', 'index.control')]), ]   # c = combine, concatanate 之意思

In [77]:
# stddiff 標準化差; stddiff.I, stddiff.u 代表 95% 信賴區間
std_diff(matched, 'married', 'treat')

[1] -0.02723636

In [78]:
# Q5
# For the propensity score matched data: Which variable has the largest standardized difference?
lapply(c('age', 'nodegree', 're74', 'black'), function(v) std_diff(matched, v, 'treat'))

[[1]]
[1] 0.05837946

[[2]]
[1] 0.1272087

[[3]]
[1] -0.05612874

[[4]]
[1] 0.8518868

In [65]:
# Q6
# Re-do the matching, but use a caliper this time. Set the caliper=0.1 in the options in the Match function. Again, before running the Match function, set the seed: 931139
# How many matched pairs are there? 有幾對
# caliper -- 去除極端值
set.seed(931139)
pmatch <- Match(Tr = lalonde$treat, M=1, X=lalonde$pscore, replace=FALSE, caliper=0.1)
matched <- lalonde[unlist(pmatch[c('index.treated', 'index.control')]), ]

In [66]:
nrow(matched)/2 

[1] 111

In [68]:
# Q7
# Use the matched data set (from propensity score matching with caliper=0.1) to carry out the outcome analysis.
# For the matched data, what is the mean of real earnings in 1978 for treated subjects minus the mean of real earnings in 1978 for untreated subjects?
mean(matched[matched$treat==1,]$re78) - mean(matched[matched$treat==0,]$re78)

[1] 1246.806

In [69]:
# Q8
# Use the matched data set (from propensity score matching with caliper=0.1) to carry out the outcome analysis.
# Carry out a paired t-test for the effect of treatment on earnings. What are the values of the 95% confidence interval?
t.test(matched[matched$treat==1,c('re78')],
       matched[matched$treat==0,c('re78')],
       conf.level = 0.95, paired=TRUE)


	Paired t-test

data:  matched[matched$treat == 1, c("re78")] and matched[matched$treat == 0, c("re78")]
t = 1.4824, df = 110, p-value = 0.1411
alternative hypothesis: true difference in means is not equal to 0
95 percent confidence interval:
 -420.0273 2913.6398
sample estimates:
mean of the differences 
               1246.806 


In [ ]:
# <- : alt + -
# <- 符號相當於Pytohn的=(賦值的意思)